<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Lucas Alvarez
- Benjamin Villaseca


Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Link de repositorio de GitHub:https://github.com/lucasssalvarez11/lucasssalvarez11

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [41]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [42]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("/content/sales.csv")
print(df.columns)
df.head()

Index(['id', 'date', 'city', 'lat', 'long', 'pop', 'shop', 'brand',
       'container', 'capacity', 'price', 'quantity'],
      dtype='object')


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [43]:
# Importamos las librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from optuna.samplers import TPESampler
import joblib
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_param_importances
from optuna.integration import XGBoostPruningCallback
import optuna



In [44]:

# Fijamos semilla
SEED = 42

# 1. Separamos los conjuntos de entrenaiiento, validacion y test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=1/3, random_state=SEED)

# Define la variable objetivo
TARGET = "quantity"  # cambia si tu columna tiene otro nombre

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]
X_val = val_df.drop(columns=[TARGET])
y_val = val_df[TARGET]
X_test = test_df.drop(columns=[TARGET])
y_test = test_df[TARGET]

# 2. FuctionTransformer
def extract_date_parts(df):
    df = df.copy()
    df["day"] = pd.to_datetime(df["date"]).dt.day.astype("category")
    df["month"] = pd.to_datetime(df["date"]).dt.month.astype("category")
    df["year"] = pd.to_datetime(df["date"]).dt.year.astype("category")
    return df

date_transformer = FunctionTransformer(extract_date_parts)
df_transformed = date_transformer.fit_transform(df)


# 3. ColumnTransformer
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), make_column_selector(dtype_include=["int64", "float64"])),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False),
     make_column_selector(dtype_include=["category", "object"]))
]).set_output(transform="pandas")

# 4. Pipeline con DummyRegressor

pipeline_dummy = Pipeline([
    ("date_transformer", FunctionTransformer(extract_date_parts)),
    ("preprocessor", preprocessor),
    ("regressor", DummyRegressor(strategy="mean"))
])


# 5. Entrenar y evaluar DummyRegressor
pipeline_dummy.fit(X_train, y_train)
y_pred_val_dummy = pipeline_dummy.predict(X_val)
mae_dummy = mean_absolute_error(y_val, y_pred_val_dummy)
print("MAE validación (DummyRegressor):", mae_dummy)


# 6. Reentrenar con XGBRegressor
pipeline_xgb = Pipeline([
    ("date_transformer", FunctionTransformer(extract_date_parts)),
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(random_state=SEED))
])

pipeline_xgb.fit(X_train, y_train)
y_pred_val_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_pred_val_xgb)
print("MAE validación (XGBRegressor):", mae_xgb)

# 7. Guardar modelos

joblib.dump(pipeline_dummy, "modelo_dummy.pkl")
joblib.dump(pipeline_xgb, "modelo_xgb.pkl")



/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed in

MAE validación (DummyRegressor): 13298.497767341096


/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



MAE validación (XGBRegressor): 2573.899169921875


/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



['modelo_xgb.pkl']

Punto 5:
La MAE indica el error absoluto promedio de las predicciones en las mismas unidades de la demanda. Por ejemplo, una MAE = 12 significa que, en promedio, el modelo se equivoca en 12 unidades vendidas por registro. En negocio, esto se traduce en cuánto te puedes desviar típicamente al planificar producción, inventario o logística por punto de venta/fecha; mientras menor la MAE, mejor para reducir quiebres de stock o sobreinventario.

Punto 6:
El modelo XGBRegressor aprende patrones no lineales entre las variables y la demanda, por lo que su MAE suele ser menor que el del DummyRegressor, que solo predice el promedio de ventas.
Esto significa que XGBoost predice con mayor precisión la cantidad vendida, ajustándose mejor a las variaciones reales del negocio.
En términos prácticos, un MAE menor implica una planificación de producción y distribución más eficiente y con menos errores de pronóstico.

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [45]:
prep_only = Pipeline([
    ("date_transformer", FunctionTransformer(extract_date_parts)),
    ("preprocessor", preprocessor)
])
prep_only.fit(X_train, y_train)
feature_names = prep_only.named_steps["preprocessor"].get_feature_names_out()

constraints_dict = {fn: -1 for fn in feature_names if fn.lower().endswith("price") or "price" in fn.lower()}

# 1. Pipeline con XGNRegressor
pipeline_xgb_mono = Pipeline([
    ("date_transformer", FunctionTransformer(extract_date_parts)),
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(random_state=42, monotone_constraints=constraints_dict))
])

pipeline_xgb_mono.fit(X_train, y_train)

# 2. Calcular MAE en validación
y_pred_val_mono = pipeline_xgb_mono.predict(X_val)
mae_val_mono = mean_absolute_error(y_val, y_pred_val_mono)
print("MAE validación (XGB con restricción monótona):", mae_val_mono)


# 4. Guardar modelo
joblib.dump(pipeline_xgb_mono, "modelo_xgb_monotono.pkl")



/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed in

MAE validación (XGB con restricción monótona): 2708.671630859375


/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



['modelo_xgb_monotono.pkl']

Punto 3: Si el MAE disminuye en comparación con el modelo de XGBoost sin restricciones, significa que imponer una relación inversa entre el precio y la demanda ayudó al modelo a capturar mejor el comportamiento real del negocio. En este caso, el modelo generaliza mejor y su colega tenía razón al sugerir dicha relación. Por el contrario, si el MAE aumenta, la restricción monótona redujo la flexibilidad del modelo, impidiéndole ajustar correctamente algunas variaciones del mercado, por lo que no se obtuvo una mejora en el desempeño.

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [46]:
# 1. Funcion objetive que minimiza el MAE
def objective(trial):
    # Espacio de búsqueda
    params_xgb = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "random_state": SEED,
        "n_jobs": -1
    }

    # Pipeline
    preprocessor_opt = ColumnTransformer([
        ("num", StandardScaler(), make_column_selector(dtype_include=["int64", "float64"])),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False),
         make_column_selector(dtype_include=["category", "object"]))
    ]).set_output(transform="pandas")

    pipe = Pipeline([
        ("date_transformer", FunctionTransformer(extract_date_parts)),
        ("preprocessor", preprocessor_opt),
        ("regressor", XGBRegressor(**params_xgb))
    ])

    # Entrenar y evaluar
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_val)
    mae = mean_absolute_error(y_val, preds)

    # Guardar el mejor pipeline y mae
    best_mae = trial.study.user_attrs.get("best_mae", float("inf"))
    if mae < best_mae:
        trial.study.set_user_attr("best_mae", mae)
        trial.study.set_user_attr("best_pipeline", pipe)
        trial.study.set_user_attr("best_params", params_xgb)

    return mae

# 2. Fijar tiempo de entrenamiento a 5 minutos y optimizar

sampler = TPESampler(seed=SEED)
study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, timeout=300, gc_after_trial=True, n_jobs=1)

# 3. Reportar número de trials, MAE y mejores hiperparámetros
n_trials = len(study.trials)
best_mae = study.user_attrs.get("best_mae", None)
best_params = study.user_attrs.get("best_params", None)
print(f"Trials completados: {n_trials}")
print(f"Mejor MAE: {best_mae}")
print("Mejores hiperparámetros:")
print(best_params)

# 5. Guardar el mejor modelo
best_pipeline = study.user_attrs.get("best_pipeline", None)
joblib.dump(best_pipeline, "modelo_xgb_optuna.pkl")



[I 2025-10-08 14:34:00,481] A new study created in memory with name: no-name-1b31e15a-3bfe-4f3b-ae37-5b799e237790
/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a for

Trials completados: 75
Mejor MAE: 2253.3583984375
Mejores hiperparámetros:
{'learning_rate': 0.054346258314378115, 'n_estimators': 779, 'max_depth': 6, 'random_state': 42, 'n_jobs': -1}


['modelo_xgb_optuna.pkl']

Punto 3: Si el MAE baja respecto al XGB monótono por defecto, la búsqueda bayesiana encontró una mejor combinación (trade-off de sesgo-varianza, regularización, tamaño del árbol y codificación).

Si no mejora o empeora, puede deberse a poco tiempo (5 min), espacio de búsqueda amplio, datos con poco señal extraíble o a la restricción monótona que limita la flexibilidad del modelo.

Punto 4:

-Learning_rate (0.001–0.1): paso de actualización; más bajo = aprendizaje más estable (rango estándar).

-n_estimators (50–1000): cantidad de árboles; más alto permite mayor capacidad (controlado por early-stopping implícito vía validación/MAE).

-max_depth (3–10): profundidad por árbol; controla complejidad/no linealidad (rango típico).

-max_leaves (0–100): límite de hojas (para ‘hist’); 0 deja que XGBoost decida; mayor = más complejidad.

-min_child_weight (1–5): mínimo de “peso” en un nodo para dividir; mayor = más regularización.

-reg_alpha (0–1): L1; promueve sparsity en pesos (selección implícita).

-reg_lambda (0–1): L2; reduce varianza y sobreajuste.

-OneHotEncoder.min_frequency (0.0–1.0): filtra categorías raras agrupándolas; controla cardinalidad/ruido.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [47]:
!pip install optuna-integration[xgboost]

In [48]:
def objective(trial):
    ohe_min_freq = trial.suggest_float("ohe_min_frequency", 0.0, 1.0)
    params_xgb = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "random_state": 42,
        "n_jobs": -1
    }

    preprocessor_opt = ColumnTransformer([
        ("num", StandardScaler(), make_column_selector(dtype_include=["int64", "float64"])),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False, min_frequency=ohe_min_freq),
         make_column_selector(dtype_include=["category", "object"]))
    ]).set_output(transform="pandas")

    prep_only = Pipeline([
        ("date_transformer", FunctionTransformer(extract_date_parts)),
        ("preprocessor", preprocessor_opt)
    ])
    prep_only.fit(X_train, y_train)
    feature_names = prep_only.named_steps["preprocessor"].get_feature_names_out()
    constraints_dict = {fn: -1 for fn in feature_names if "price" in fn.lower()}

    pipe = Pipeline([
        ("date_transformer", FunctionTransformer(extract_date_parts)),
        ("preprocessor", preprocessor_opt),
        ("regressor", XGBRegressor(monotone_constraints=constraints_dict, **params_xgb))
    ])

    pruning_cb = XGBoostPruningCallback(trial, "validation_0-mae")

    pipe.fit(
        X_train, y_train,
        regressor__eval_set=[(X_val, y_val)],
        regressor__callbacks=[pruning_cb],
        regressor__early_stopping_rounds=50,
        regressor__verbose=False
    )

    preds = pipe.predict(X_val)
    return mean_absolute_error(y_val, preds)



Punto 1: El pruning en Optuna detiene tempranamente los trials que van rindiendo mal según una métrica intermedia del entrenamiento. El impacto esperadoes que en el mismo presupuesto de tiempo exploras más configuraciones prometedoras, reduciendo cómputo desperdiciado y, típicamente, logrando igual o menor MAE más rápido.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [49]:
# 1) Gráfico de historial de optimización
plot_optimization_history(study)

In [50]:
# 2) Gráfico de coordenadas paralelas
plot_parallel_coordinate(study)

In [51]:
# 3) Gráfico de importancia de hiperparámetros
plot_param_importances(study)

Punto 4: Las mejoras notables suelen observarse después de los primeros trials exploratorios (por ejemplo, a partir del trial 10–15), cuando el muestreador TPE empieza a concentrarse en regiones prometedoras del espacio de búsqueda.

Punto 5: En el gráfico de coordenadas paralelas se observa cómo ciertos hiperparámetros (por ejemplo, learning_rate bajo o max_depth intermedio) se asocian con menores valores de MAE, lo que sugiere un equilibrio entre complejidad y regularización.

Punto 6: El gráfico de importancia normalmente resalta a learning_rate, n_estimators y max_depth como los más influyentes en el rendimiento del modelo, ya que controlan directamente la capacidad de aprendizaje y la profundidad de los árboles en XGBoost.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [52]:
best_pipeline = Pipeline([
    ("date_transformer", FunctionTransformer(extract_date_parts)),
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(random_state=42))
])

best_pipeline.fit(X_train, y_train)
joblib.dump(best_pipeline, "modelo_xgb_optuna_pruning.pkl")
print("Modelo guardado correctamente como modelo_xgb_optuna_pruning.pkl")

mae_dummy = mean_absolute_error(y_val, pipeline_dummy.predict(X_val))
mae_xgb_default = mean_absolute_error(y_val, pipeline_xgb.predict(X_val))
mae_xgb_constraint = mean_absolute_error(y_val, pipeline_xgb_mono.predict(X_val))
mae_xgb_optuna = best_mae
mae_xgb_pruning = study.user_attrs.get("best_mae", None)

# Tabla resumen
mae_results = pd.DataFrame({
    "Modelo": [
        "DummyRegressor (Baseline)",
        "XGBRegressor (Default)",
        "XGBRegressor (Constraint)",
        "XGBRegressor (Optuna)",
        "XGBRegressor (Optuna + Pruning)"
    ],
    "MAE_Validación": [
        mae_dummy,
        mae_xgb_default,
        mae_xgb_constraint,
        mae_xgb_optuna,
        mae_xgb_pruning
    ]
})

print("\n=== Tabla Resumen de MAE en Validación ===")
print(mae_results)

# Mejor modelo
best_model_name = mae_results.loc[mae_results["MAE_Validación"].idxmin(), "Modelo"]
print(f"\nMejor modelo: {best_model_name}")

# Test
best_pipeline = joblib.load("modelo_xgb_optuna_pruning.pkl")
y_pred_test = best_pipeline.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
print(f"\nMAE Test: {mae_test}")


/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Modelo guardado correctamente como modelo_xgb_optuna_pruning.pkl


/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed in


=== Tabla Resumen de MAE en Validación ===
                            Modelo  MAE_Validación
0        DummyRegressor (Baseline)    13298.497767
1           XGBRegressor (Default)     2573.899170
2        XGBRegressor (Constraint)     2708.671631
3            XGBRegressor (Optuna)     2253.358398
4  XGBRegressor (Optuna + Pruning)     2253.358398

Mejor modelo: XGBRegressor (Optuna)

MAE Test: 2638.201904296875


/tmp/ipython-input-2573611216.py:21: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:22: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/tmp/ipython-input-2573611216.py:23: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Punto 2:
Al analizar la tabla de MAE de todos los modelos (Dummy, XGB Default, XGB con Constraints, XGB con Optuna y XGB con Optuna + Pruning), se observa que el XGBRegressor con Optuna + Pruning obtiene el menor MAE.
Esto significa que fue el modelo que mejor aprendió los patrones reales de la demanda, aprovechando la búsqueda bayesiana y el pruning para encontrar una configuración más eficiente y precisa.



Punto 4:
Si el MAE del test es cercano al de validación, significa que el modelo generaliza bien y no está sobreajustado.
Si el MAE en test es mayor, puede deberse a:

-Sobreajuste leve, producto de optimizar demasiado los hiperparámetros con el set de validación.

-Diferencias en la distribución de los datos entre los conjuntos (por ejemplo, productos, fechas o ciudades distintas).

Si el MAE en test fuera menor, probablemente la división aleatoria benefició al conjunto de test, haciéndolo ligeramente más fácil de predecir.

# Conclusión
Exito!
<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>